In [ ]:
from google.colab import drive
drive.mount('/content/shared', force_remount=True)

In [ ]:
!unzip /content/shared/MyDrive/images.zip

In [ ]:
!git clone https://github.com/OmarFarag95/minimum-area-bounding-rectangle-python3.git min_bbox

In [ ]:
!pip install imgaug

In [ ]:
# import the necessary packages
import os
# define the base path to the input dataset and then use it to derive
# the path to the images directory and annotation CSV file
BASE_PATH = "/content"
IMAGES_PATH = os.path.sep.join([BASE_PATH, "images"])
ANNOTS_PATH = os.path.sep.join([BASE_PATH, "metadata.json"])

In [ ]:
# define the path to the base output directory
BASE_OUTPUT = "output"
if not os.path.exists(BASE_OUTPUT):
    os.makedirs(BASE_OUTPUT)
# define the path to the output serialized model, model training plot,
# and testing image filenames
MODEL_PATH = os.path.sep.join([BASE_OUTPUT, "detector.h5"])
PLOT_PATH = os.path.sep.join([BASE_OUTPUT, "plot.png"])
TEST_FILENAMES = os.path.sep.join([BASE_OUTPUT, "test_images.txt"])

In [ ]:
# initialize our initial learning rate, number of epochs to train
# for, and the batch size
INIT_LR = 1e-4
NUM_EPOCHS = 2
BATCH_SIZE = 16

In [ ]:
# import the necessary packages
from tensorflow.keras.applications import VGG16, resnet
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os

In [ ]:
!cp shared/MyDrive/metadata.json /content/

In [ ]:
def get_bounding_box(points):

    bot_left_x = min(point[0] for point in points)
    bot_left_y = min(point[1] for point in points)
    top_right_x = max(point[0] for point in points)
    top_right_y = max(point[1] for point in points)

    return [(bot_left_x, bot_left_y), (top_right_x, top_right_y)]

In [ ]:
from numpy import *

from min_bbox.python.qhull_2d import *
from min_bbox.python.min_bounding_rect import *

def bounding_box_v2(points):
    
    xy_points = array(points)

    hull_points = qhull2D(xy_points)

    # Reverse order of points, to match output from other qhull implementations
    hull_points = hull_points[::-1]

    #print ('Convex hull points: \n', hull_points, "\n")

    # Find minimum area bounding rectangle
    (rot_angle, area, width, height, center_point, corner_points) = minBoundingRect(hull_points)

    return corner_points

In [ ]:
v = [[246.09293140559757, 222.41169803156015],[250.44603729904748, 211.9438727322957],
    [283.09433149992174, 279.3110343649168],
    [297.5046130999609, 244.07283583161148],
    [214.5704404561325, 249.46220866382959],
    [233.4088814757176, 204.68894331699937]]

bounding_box_v2(v)

In [ ]:
# load the contents of the json annotations file
print("[INFO] loading dataset...")

import json
with open('metadata.json') as f:
  metadata = json.load(f)

rows = []
set_sizes = []
for k,v in metadata.items():
    tuples_bounds = []
    xy_tuples = list(set([(int(sample['x']),int(sample['y'])) for sample in v["bounds_x_y"]]))
    ## get boundarybox
    if(len(xy_tuples)<4):
      continue
    bbox = bounding_box_v2(xy_tuples)
    rows.append({k:bbox.tolist()})
  
# initialize the list of data (images), our target output predictions
# (bounding box coordinates), along with the filenames of the
# individual images
data = []
targets = []
filenames = []

In [ ]:
##augmentation
import random
random.seed(0)
import imgaug as ia
import imgaug.augmenters as iaa

#select 1000 random images

random_images = random.choices(rows,k=1000)
c=0
for image_info in random_images:
  for k, v in image_info.items():

    polygons = [[ia.Polygon(v)]]

    imagePath = os.path.sep.join([IMAGES_PATH, k])
    image = cv2.imread(imagePath)

    if image_info in rows:
      index = rows.index(image_info)

      if image is not None:
      
        seq = iaa.CropAndPad(
                  percent=(-0.15, 0.5),
                  pad_mode=ia.ALL,
                  pad_cval=(0, 0))
        
        images_aug, polygons_aug = seq(image=image, polygons=polygons)
        x1,x2,x3,x4 = polygons_aug[0][0].xx
        y1,y2,y3,y4 = polygons_aug[0][0].yy

        new_coordinates = {k:[(int(x1),int(y1)),(int(x2),int(y2)),[int(x3),int(y3)],[int(x4),int(y4)]]}

        # override image in its original path and update its coordinates in rows
        cv2.imwrite(f"IMAGES_PATH/{k}", images_aug)
        
        rows[index] = new_coordinates

        c+=1

print(c)

In [ ]:
with open('pools_ground_truth.json','w') as f:
  json.dump(rows,f)

In [ ]:
	# derive the path to the input image, load the image (in OpenCV
	# format), and grab its dimensions
from tqdm import tqdm
for r in tqdm(range(0,len(rows),1)):
  example = rows[r]
  for k,v in example.items():
    filename = k
    imagePath = os.path.sep.join([IMAGES_PATH, filename])
    image = cv2.imread(imagePath)
    if image is not None:
      (h, w) = image.shape[:2]
      # scale the bounding box coordinates relative to the spatial
      # dimensions of the input image

      p_1, p_2, p_3, p_4 = list(v)
      x1 = p_1[0]/w
      y1 = p_1[1]/h

      x2 = p_2[0]/w
      y2 = p_2[1]/h
      
      x3 = p_3[0]/w
      y3 = p_3[1]/h
      
      x4 = p_4[0]/w
      y4 = p_4[1]/h
      
    
      image = load_img(imagePath, target_size=(150, 150))
      image = img_to_array(image, dtype=np.uint8)
      # update our list of data, targets, and filenames
      data.append(image)
      corners = [x1, y1, x2, y2, x3, y3 , x4, y4]
      targets.append(corners)
      filenames.append(filename)

In [ ]:
split = train_test_split(data, targets, filenames, test_size=0.10,
	random_state=42)

In [ ]:
(trainImages, testImages) = split[:2]
(trainTargets, testTargets) = split[2:4]
(trainFilenames, testFilenames) = split[4:]

In [ ]:
trainImages = np.array(trainImages,dtype="float32") /255.0

In [ ]:
testImages = np.array(testImages,dtype="float32") /255.0

In [ ]:
trainTargets = np.array(trainTargets, dtype="float32")

In [ ]:
testTargets = np.array(testTargets, dtype="float32")

In [ ]:
data = []
split = []
targets = []
filenames = []
rows = []

In [ ]:
# data = np.array(data,dtype="float32") /255.0
# targets = np.array(targets, dtype="float32")
# # partition the data into training and testing splits using 90% of
# # the data for training and the remaining 10% for testing
# split = train_test_split(data, targets, filenames, test_size=0.20,
# 	random_state=42)
# unpack the data split

# write the testing filenames to disk so that we can use then
# when evaluating/testing our bounding box regressor
print('Training size:', len(trainImages))
print('Test size:', len(testImages))

In [ ]:
print("[INFO] saving testing filenames...")
f = open(TEST_FILENAMES, "w")
f.write("\n".join(testFilenames))
f.close()

In [ ]:
data = []

In [ ]:
# load the VGG16 network, ensuring the head FC layers are left off
# vgg = VGG16(weights="imagenet", include_top=False,
# 	input_tensor=Input(shape=(150, 150, 3)))

resnetn = resnet.ResNet101(
    include_top=False,
    weights='imagenet',
    input_tensor=Input(shape=(150, 150, 3))
)

# freeze all VGG layers so they will *not* be updated during the
# training process
resnetn.trainable = False
# flatten the max-pooling output of VGG
flatten = resnetn.output
flatten = Flatten()(flatten)
# construct a fully-connected layer header to output the predicted
# bounding box coordinates
bboxHead = Dense(128, activation="relu")(flatten)
bboxHead = Dense(64, activation="relu")(flatten)
bboxHead = Dense(8, activation="sigmoid")(bboxHead)
# construct the model we will fine-tune for bounding box regression
model = Model(inputs=resnetn.input, outputs=bboxHead)

In [ ]:
!pip install tensorflow_addons
import tensorflow_addons as tfa

In [ ]:
# initialize the optimizer, compile the model, and show the model
# summary
opt = SGD(lr=INIT_LR)
model.compile(loss="mse", optimizer=opt)
print(model.summary())
# train the network for bounding box regression
print("[INFO] training bounding box regressor...")
H = model.fit(
	trainImages, trainTargets,
	validation_data=(testImages, testTargets),
	batch_size=1,
	epochs=3,
	verbose=1)

In [ ]:
# serialize the model to disk
model.save(MODEL_PATH, save_format="h5")
# plot the model training history
N = 3
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.title("Bounding Box Regression Loss on Training Set")
plt.xlabel("Epoch #")
plt.ylabel("Loss")
plt.legend(loc="lower left")
plt.savefig(PLOT_PATH)

In [ ]:
def bb_intersection_over_union(boxA, boxB):
	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(boxA[0], boxB[0])
	yA = max(boxA[1], boxB[1])
	xB = min(boxA[2], boxB[2])
	yB = min(boxA[3], boxB[3])
	# compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
	# compute the area of both the prediction and ground-truth
	# rectangles
	boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
	boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
	# compute the intersection over union by taking the intersection
	# area and dividing it by the sum of prediction + ground-truth
	# areas - the interesection area
	iou = interArea / float(boxAArea + boxBArea - interArea)
	# return the intersection over union value
	return iou

In [ ]:
from tensorflow.keras.models import load_model
model = load_model('/content/output/detector.h5')

In [ ]:

with open('pools_ground_truth.json','r') as f:
  ground_truth = json.load(f)

def predict(imagePath):
  image = load_img(imagePath, target_size=(150, 150))
  image = img_to_array(image) / 255.0
  image = np.expand_dims(image, axis=0)
    # make bounding box predictions on the input image
  preds = model.predict(image)[0]
  (x1,y1,x2,y2,x3,y3,x4,y4) = preds
  # load the input image (in OpenCV format), resize it such that it
  # fits on our screen, and grab its dimensions
  image = cv2.imread(imagePath)
  #image = imutils.resize(image, width=600)
  (h, w) = image.shape[:2]
  # scale the predicted bounding box coordinates based on the image
  # dimensions

  ## plot predicted bbox
  x1 = x1*w
  y1 = y1*h
  x2 = x2*w
  y2 = y2*h
  x3 = x3*w
  y3 = y3*h
  x4 = x4*w
  y4 = y4*h

  #pts1 = np.array([[x1,y1],[x2,y2],[x3,y3],[x4,y4]], np.int32)

  pts1 = [[x1,y1],[x2,y2],[x3,y3],[x4,y4]]
  
  #pts1 = pts1.reshape((-1,1,2))

  #cv2.polylines(image,[pts],True,(0,255,255))



  ## plot original bbox
  for t in ground_truth:
    for k, v in t.items():
      if k == imagePath.split('/')[1]:
        coors = v
  x1,y1 = coors[0][0], coors[0][1]
  x2,y2 = coors[1][0], coors[1][1]
  x3,y3 = coors[2][0], coors[2][1]
  x4,y4 = coors[3][0], coors[3][1]



  pts2 = [[x1,y1],[x2,y2],[x3,y3],[x4,y4]]
  
  return np.array(get_bounding_box(pts1)).flatten(), np.array(get_bounding_box(pts2)).flatten()

In [ ]:
scores = []
with open("output/test_images.txt", 'r') as f:
  for line in f:
    pts1,pts2 = predict(f"images/{line.strip()}")
    v = bb_intersection_over_union(pts1, pts2)
    scores.append(v)

In [ ]:

import numpy as np
from scipy.stats import binned_statistic

data = scores
bin_means = binned_statistic(data, data, bins=10, range=(0, 1))[0]

In [ ]:
plt.figure(figsize=(10,8))
plt.hist(scores, density=False, bins=10)  # density=False would make counts
plt.ylabel('Counts')
plt.xlabel('IoU');
plt.savefig('iou_metric.png')